In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [275]:
import datetime

In [276]:
import warnings
warnings.filterwarnings('ignore')

In [277]:
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
book_tags = pd.read_csv('book_tags.csv')
tags = pd.read_csv('tags.csv')

In [278]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [279]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [280]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [281]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [282]:
books.shape,ratings.shape,book_tags.shape,tags.shape

((10000, 23), (5976479, 3), (999912, 3), (34252, 2))

In [283]:
ratings=ratings.drop_duplicates()

In [284]:
unwanted_users = ratings.groupby('user_id')['user_id'].count()
unwanted_users = unwanted_users[unwanted_users < 30]

In [285]:
unwanted_users

user_id
9256     26
9727     29
10369    28
11286    29
11620    27
         ..
50814    25
50918    22
51174    25
51725    21
52083    25
Name: user_id, Length: 153, dtype: int64

In [286]:
unwanted_rating=ratings[ratings.user_id.isin(unwanted_users.index)]

In [287]:
unwanted_rating.shape

(4003, 3)

In [288]:
ratings.drop(unwanted_rating.index,inplace=True)

In [289]:
ratings.shape

(5972476, 3)

In [290]:
book_name=books[['book_id','title']]

In [291]:
new_rating=pd.merge(ratings,book_name,how='left',on='book_id',sort=True)

In [292]:
new_rating

,user_id,book_id,rating,title
0,2886,1,5,"The Hunger Games (The Hunger Games, #1)"
1,6158,1,5,"The Hunger Games (The Hunger Games, #1)"
2,3991,1,4,"The Hunger Games (The Hunger Games, #1)"
3,5281,1,5,"The Hunger Games (The Hunger Games, #1)"
4,5721,1,5,"The Hunger Games (The Hunger Games, #1)"
...,...,...,...,...
5972471,35336,10000,4,The First World War
5972472,17999,10000,3,The First World War
5972473,49007,10000,4,The First World War
5972474,43319,10000,5,The First World War


In [293]:
books=books.dropna(subset=['original_publication_year'])

In [294]:
books['original_publication_year']=books['original_publication_year'].astype(int)

In [295]:
books_copy=books
books_copy_2=books

In [296]:
year=2000
books=books[books['original_publication_year']>year]

In [297]:
#50000
m=books['ratings_count'].quantile(0.80)

In [298]:
#[books['ratings_count']>books['ratings_count'].quantile(0.95)]
v=books[books['ratings_count']>books['ratings_count'].quantile(0.80)]['ratings_count']

In [299]:
#[books['ratings_count']>books['ratings_count'].quantile(0.95)]
R = books[books['ratings_count']>books['ratings_count'].quantile(0.80)]['average_rating']
C = books['average_rating'].mean()
W = (R*v + C*m) / (v + m)

In [300]:
top_250=W.sort_values(ascending=False).head(250)

In [301]:
index_list=list(top_250.index)

In [302]:
best_books_250=books.loc[index_list, ['book_id','original_title','authors','original_publication_year','average_rating']]

In [303]:
recoomdation=pd.concat([best_books_250,top_250],axis=1)

In [304]:
recoomdation.rename(columns ={0:'weighted_rating'},inplace = True)

In [305]:
recoomdation=recoomdation.reset_index()
recoomdation

,index,book_id,original_title,authors,original_publication_year,average_rating,weighted_rating
0,24,25,Harry Potter and the Deathly Hallows,"J.K. Rowling, Mary GrandPré",2007,4.61,4.594178
1,26,27,Harry Potter and the Half-Blood Prince,"J.K. Rowling, Mary GrandPré",2005,4.54,4.525406
2,1307,1308,A Court of Mist and Fury,Sarah J. Maas,2016,4.72,4.503255
3,191,192,The Name of the Wind,Patrick Rothfuss,2007,4.55,4.492509
4,561,562,The Way of Kings,Brandon Sanderson,2010,4.64,4.483884
...,...,...,...,...,...,...,...
245,1850,1851,The Emperor of All Maladies: A Biography of Ca...,Siddhartha Mukherjee,2010,4.29,4.142941
246,2479,2480,Into the Wild,Erin Hunter,2003,4.29,4.142932
247,1133,1134,Gabriel's Rapture,Sylvain Reynard,2012,4.22,4.142596
248,675,676,Entwined with You,Sylvia Day,2013,4.19,4.140748


In [306]:
pd.merge(book_tags,tags,on='tag_id',how='left')

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,1,11305,37174,fantasy
2,1,11557,34173,favorites
3,1,8717,12986,currently-reading
4,1,33114,12716,young-adult
...,...,...,...,...
999907,33288638,21303,7,neighbors
999908,33288638,17271,7,kindleunlimited
999909,33288638,1126,7,5-star-reads
999910,33288638,11478,7,fave-author


In [307]:
genres = ["Art", "Biography", "Business", "Chick-Lit", "Children-s", "Christian", "Classics",
          "Comics", "Contemporary", "Cookbooks", "Crime", "Ebooks", "Fantasy", "Fiction",
          "Gay-and-Lesbian", "Graphic-Novels", "Historical-Fiction", "History", "Horror",
          "Humor-and-Comedy", "Manga", "Memoir", "Music", "Mystery", "Nonfiction", "Paranormal",
          "Philosophy", "Poetry", "Psychology", "Religion", "Romance", "Science", "Science-Fiction", 
          "Self-Help", "Suspense", "Spirituality", "Sports", "Thriller", "Travel", "Young-Adult"]

In [308]:
genres = list(map(str.lower, genres))

In [309]:
available_genres = tags.loc[tags.tag_name.str.lower().isin(genres)]

In [310]:
available_genres.shape

(40, 2)

In [311]:
genres=pd.Series(genres)

In [312]:
len(genres)

40

In [313]:
book_genre_coll=pd.merge(book_tags,available_genres,on='tag_id')

In [314]:
book_genre_coll.drop(['tag_id','count'],axis=1,inplace=True)

In [315]:
book_genre_coll

,goodreads_book_id,tag_name
0,1,fantasy
1,2,fantasy
2,3,fantasy
3,5,fantasy
4,6,fantasy
...,...,...
73330,165395,gay-and-lesbian
73331,272315,gay-and-lesbian
73332,595375,gay-and-lesbian
73333,23316,humor-and-comedy


In [316]:
books_copy['authors']=books_copy['authors'].str.replace(" ","")

In [317]:
books_copy['authors']=books_copy['authors'].str.replace(","," ")

In [318]:
books_copy['authors']=books_copy['authors'].str.lower()

In [319]:
books_copy.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

In [320]:
books_copy=books_copy[['book_id','goodreads_book_id','authors','original_publication_year','original_title','average_rating','ratings_count']]

In [321]:
books_copy

,book_id,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_count
0,1,2767052,suzannecollins,2008,The Hunger Games,4.34,4780653
1,2,3,j.k.rowling marygrandpré,1997,Harry Potter and the Philosopher's Stone,4.44,4602479
2,3,41865,stepheniemeyer,2005,Twilight,3.57,3866839
3,4,2657,harperlee,1960,To Kill a Mockingbird,4.25,3198671
4,5,4671,f.scottfitzgerald,1925,The Great Gatsby,3.89,2683664
...,...,...,...,...,...,...,...
9995,9996,7130616,ilonaandrews,2010,Bayou Moon,4.09,17204
9996,9997,208324,roberta.caro,1990,Means of Ascent,4.25,12582
9997,9998,77431,patricko'brian,1977,The Mauritius Command,4.35,9421
9998,9999,8565083,peggyorenstein,2011,Cinderella Ate My Daughter: Dispatches from th...,3.65,11279


In [322]:
df=pd.merge(books_copy,book_genre_coll,on='goodreads_book_id',how='left')

In [323]:
df=df[df.tag_name!='ebooks']

In [324]:
df=df.groupby(['book_id','goodreads_book_id','authors','original_publication_year','original_title','average_rating','ratings_count'], as_index=False).agg({'tag_name' : ', '.join})

In [325]:
df['tag_name']=df['tag_name'].str.replace(","," ")

In [326]:
df['original_title']=df['original_title'].str.lower()

In [327]:
df['cont_base']=df['original_title']+" "+df['authors']+" "+df['tag_name']

In [328]:
df=df[df.cont_base.map(lambda x: x.isascii())]

In [329]:
df=df.reset_index()

In [330]:
df=df.drop(['index'],axis=1)

In [331]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8751 entries, 0 to 8750
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    8751 non-null   int64  
 1   goodreads_book_id          8751 non-null   int64  
 2   authors                    8751 non-null   object 
 3   original_publication_year  8751 non-null   int32  
 4   original_title             8751 non-null   object 
 5   average_rating             8751 non-null   float64
 6   ratings_count              8751 non-null   int64  
 7   tag_name                   8751 non-null   object 
 8   cont_base                  8751 non-null   object 
dtypes: float64(1), int32(1), int64(3), object(4)
memory usage: 581.2+ KB


In [332]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [333]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['cont_base'])

In [334]:
len(count.get_feature_names_out())

40210

In [335]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [336]:
len(cosine_sim)

8751

In [337]:
indices = pd.Series(df.index, index=df['original_title'])

In [338]:
title='the hunger games'

In [339]:
idx=indices[title]

In [340]:
idx

0

In [341]:
score= list(enumerate(cosine_sim[idx]))

In [342]:
#function returns the 1st value of the row's list of 2
best_rank=sorted(score,reverse=True,key=lambda x: x[1])

In [343]:
top_25=best_rank[1:26]

In [344]:
top_25_name=[]
for x in range(len(top_25)):
    top_25_name.append(top_25[x][0])

In [345]:
content_fil=df.loc[top_25_name]

In [346]:
content_fil

,book_id,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_count,tag_name,cont_base
459,507,7938275,suzannecollins,2010,the hunger games box set,4.49,129665,fantasy young-adult fiction romance contem...,the hunger games box set suzannecollins fantas...
13,17,6148028,suzannecollins,2009,catching fire,4.30,1831039,fantasy young-adult fiction romance contem...,catching fire suzannecollins fantasy young-ad...
15,20,7260188,suzannecollins,2010,mockingjay,4.03,1719760,fantasy young-adult fiction romance thrill...,mockingjay suzannecollins fantasy young-adult...
6169,6915,71864,johnmarsden,1994,the dead of the night,4.13,15240,fantasy young-adult fiction romance contem...,the dead of the night johnmarsden fantasy you...
8487,9672,82553,johnmarsden,1998,the night is for hunting,4.15,10908,young-adult fiction romance contemporary t...,the night is for hunting johnmarsden young-adu...
8528,9725,17950614,nealshusterman,2014,undivided,4.48,11403,fantasy young-adult fiction romance contem...,undivided nealshusterman fantasy young-adult ...
2542,2799,13326831,joellecharbonneau,2013,the testing,4.05,38142,fantasy young-adult fiction romance thrill...,the testing joellecharbonneau fantasy young-a...
922,1008,8659601,pittacuslore,2011,the power of six,4.17,97938,fantasy young-adult fiction paranormal mys...,the power of six pittacuslore fantasy young-a...
1568,1716,12971616,pittacuslore,2012,the rise of nine,4.20,70196,fantasy young-adult fiction paranormal mys...,the rise of nine pittacuslore fantasy young-a...
6293,7057,12753231,emmylaybourne,2012,monument 14,3.89,17360,fantasy young-adult fiction romance contem...,monument 14 emmylaybourne fantasy young-adult...


In [347]:
new_rating=new_rating.sort_values(by='user_id')

In [348]:
new_rating

,user_id,book_id,rating,title
3634284,1,1521,5,"Antigone (The Theban Plays, #3)"
5490997,1,6665,4,Cry to Heaven
1742281,1,268,3,Never Let Me Go
989756,1,100,4,The Poisonwood Bible
1100119,1,119,3,The Handmaid's Tale
...,...,...,...,...
4648740,53424,3390,5,Caddie Woodlawn (Caddie Woodlawn #1)
2193791,53424,427,5,The Importance of Being Earnest
5798472,53424,8609,4,The Lady of Shalott
123461,53424,7,5,The Hobbit


In [350]:
piv_tab=new_rating.pivot(index='user_id',columns='book_id',values='rating')

In [351]:
piv_tab

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,4.0,4.0,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,4.0,5.0,3.0,NaN,2.0,NaN,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53421,4.0,5.0,NaN,5.0,4.0,NaN,4.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53422,4.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [359]:
title_user_ratings = piv_tab[3]
similar_to_title = piv_tab.corrwith(title_user_ratings)
corr_title = pd.DataFrame(similar_to_title, columns=['correlation'])
corr_title.dropna(inplace=True)
corr_title.sort_values('correlation', ascending=False, inplace=True)

In [366]:
corr_title=corr_title.reset_index().iloc[1:11]

In [367]:
corr_title

,book_id,correlation
1,9487,0.952579
2,992,0.893485
3,9878,0.893315
4,1619,0.887020
5,2021,0.869397
6,52,0.831614
7,9032,0.823055
8,8616,0.818182
9,9614,0.816161
10,49,0.815964


In [368]:
books_copy_2=books_copy_2[['book_id','authors','title','average_rating']]

In [375]:
pd.merge(corr_title,books_copy_2,on='book_id',how='left')

,book_id,correlation,authors,title,average_rating
0,9487,0.952579,michelhouellebecq,La Carte et le territoire,3.93
1,992,0.893485,stepheniemeyer ilyanakadushin mattwalters,"The Twilight Saga (Twilight, #1-4)",3.88
2,9878,0.893315,danariely,The Honest Truth About Dishonesty: How We Lie ...,3.91
3,1619,0.887020,stepheniemeyer,The Twilight Saga Complete Collection (Twilig...,4.31
4,2021,0.869397,stepheniemeyer,"The Twilight Collection (Twilight, #1-3)",3.78
5,52,0.831614,stepheniemeyer,"Eclipse (Twilight, #3)",3.69
6,9032,0.823055,andrewhunt davethomas,The Pragmatic Programmer: From Journeyman to M...,4.31
7,8616,0.818182,سلطانموسىالموسى,أقوم قيلا,3.99
8,9614,0.816161,alexberenson,"The Faithful Spy (John Wells, #1)",3.97
9,49,0.815964,stepheniemeyer,"New Moon (Twilight, #2)",3.52


In [358]:
books_copy_2

,book_id,authors,title,average_rating
0,1,suzannecollins,"The Hunger Games (The Hunger Games, #1)",4.34
1,2,j.k.rowling marygrandpré,Harry Potter and the Sorcerer's Stone (Harry P...,4.44
2,3,stepheniemeyer,"Twilight (Twilight, #1)",3.57
3,4,harperlee,To Kill a Mockingbird,4.25
4,5,f.scottfitzgerald,The Great Gatsby,3.89
...,...,...,...,...
9995,9996,ilonaandrews,"Bayou Moon (The Edge, #2)",4.09
9996,9997,roberta.caro,"Means of Ascent (The Years of Lyndon Johnson, #2)",4.25
9997,9998,patricko'brian,The Mauritius Command,4.35
9998,9999,peggyorenstein,Cinderella Ate My Daughter: Dispatches from th...,3.65
